In [ ]:
#Instance Intan connection
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
import time
from matplotlib import pyplot as plt
import numpy as np
from datetime import datetime
from pytz import timezone
base = BaseOverlay('base.bit')
PMOD_SCLK_PIN = 3
PMOD_MISO_PIN = 2
PMOD_MOSI_PIN = 1
PMOD_SS_PIN = 0
SPI_CLOCK_PHASE =0
SPI_CLOCK_PLOARITY =0
iop=base.iop_pmoda 
lib=MicroblazeLibrary(iop,['spi']) #open SPI on PmodA
spi_device=lib.spi_open(PMOD_SCLK_PIN,PMOD_MISO_PIN,PMOD_MOSI_PIN,PMOD_SS_PIN)
spi_device.configure(SPI_CLOCK_PHASE,SPI_CLOCK_PLOARITY)
rootpath = '/home/xilinx/jupyter_notebooks/'
CFactor=0.000152588
print('finish')

In [ ]:
#Load function set
def signedToUnsigned(n, byte_count):
    return int.from_bytes(n.to_bytes(byte_count, 'little', signed=True), 'little', signed=False)

def toSigned16(n):
    n = n & 0xffff
    return (n ^ 0x8000) - 0x8000

def getDateTime(): 
    format = "%Y-%m-%d %H:%M:%S"
    now_utc = datetime.now(timezone('UTC'))
    now_asia = now_utc.astimezone(timezone('Asia/Taipei'))
    print(now_asia.strftime(format))
    
def ReadIntan(reg_addr,times):
    basebit=0xC0
    bit2Send=basebit+reg_addr
    #print(bit2Send)
    byte2Send=[bit2Send,0x00]
    byte2Recv=[0x00,0x00,0x00,0x00]
    for num in range(1, times+1):
        spi_device.transfer(byte2Send,byte2Recv,len(byte2Send))
    #print(chr(byte2Recv[0]))
    MB=signedToUnsigned(byte2Recv[0], 1)
    LB=signedToUnsigned(byte2Recv[1], 1)
    realdata=MB<<8|LB
    #print(byte2Recv[1])
    return toSigned16(realdata)

def WriteIntan(R,D,timew):
    wbasebit=0x80
    wbit2Send=wbasebit+R
    #print(wbit2Send)
    wbyte2Send=[wbit2Send,D]
    wbyte2Recv=[0x00,0x00,0x00,0x00]
    for num in range(1, timew+1):
        spi_device.transfer(wbyte2Send,wbyte2Recv,len(wbyte2Send))
    #print(wbyte2Recv)
    
def CaliIntan():
    cbyte2Send=[0x55,0x00]
    cbyte2Recv=[0x00,0x00,0x00,0x00]
    spi_device.transfer(cbyte2Send,cbyte2Recv,len(cbyte2Send))
    #print(cbyte2Recv)
    
def ConvertIntan(C):
    cobyte2Send=[C,0x00]
    cobyte2Recv=[0x00,0x00,0x00,0x00]
    spi_device.transfer(cobyte2Send,cobyte2Recv,len(cobyte2Send))
    #print(cobyte2Recv)
    MB=signedToUnsigned(cobyte2Recv[0], 1)
    LB=signedToUnsigned(cobyte2Recv[1], 1)
    realdata=MB<<8|LB
    return toSigned16(realdata)*CFactor

print("Done")

In [ ]:
#Check Intan connection and should get "INTAN" 5 characters
checklist=list(range(40,45))
for rom_addr in checklist:
    readreg=ReadIntan(rom_addr,3)
    print(chr(readreg),end='')

In [ ]:
#Initial Intan Module
ReadIntan(63,2)

WriteIntan(0,0xDE,1)

WriteIntan(1,0x42,1)
WriteIntan(2,0x04,1)

WriteIntan(3,0x00,1)

WriteIntan(4,0xC0,1)

WriteIntan(5,0x40,1)
WriteIntan(6,0x80,1)
WriteIntan(7,0x00,1)

WriteIntan(8,0x16,1)
WriteIntan(9,0x80,1)
WriteIntan(10,0x17,1)
WriteIntan(11,0x80,1)

WriteIntan(12,0x2C,1)
WriteIntan(13,0x86,1)

WriteIntan(14,0x00,1)
WriteIntan(15,0xFF,1)
WriteIntan(16,0xFF,1)
WriteIntan(17,0x00,1)

CaliIntan()

ReadIntan(63,9)
print('finish')

In [ ]:
#Data name="Date Time multi_channel.txt"
#Data prefix="Date&time,invalid,invalid,CH08,CH09,CH10,CH11,CH12,CH13,CH14,CH15,CH16,CH17,CH18,CH19,CH20,CH21,CH22,CH23,invalid"
data_array=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
format = "%Y-%m-%d %H:%M:%S"
formatdata = "%Y-%m-%d %H:%M:%S.%f"
now_utc = datetime.now(timezone('UTC'))
now_asia = now_utc.astimezone(timezone('Asia/Taipei'))
filesavename=str(now_asia.strftime(format))+' multi_channel.txt'
filetosave=rootpath+filesavename
f = open(filetosave, 'w')
for led in base.leds:
        led.on()
print("start")
while (base.buttons[3].read()==0):
    now_utc = datetime.now(timezone('UTC'))
    now_asia = now_utc.astimezone(timezone('Asia/Taipei'))
    data_array[0]=str(now_asia.strftime(formatdata))+'\t'
    readlist=list(range(8,24))
    for channel2read in readlist:
        data_array[channel2read-7]=str(ConvertIntan(channel2read))+'\t'
    data_array[17]=str(ReadIntan(63,1)*CFactor)+'\t'  
    data_array[18]=str(ReadIntan(63,1)*CFactor)+'\t'
    data_array[19]=str(ReadIntan(63,1)*CFactor)+'\n'
    f.writelines(data_array)
f.close()
print("finish")
for led in base.leds:
    led.off()

In [ ]:
#Data name="Date Time single_channel [channel number].txt"
#Data prefix="Date&time,specified channel"
#Change Channel to Read Here!!!
channel_to_read=8
#Channel Range of RHD2132 is 8~23
data_array=[0,0]
format = "%Y-%m-%d %H:%M:%S"
formatdata = "%Y-%m-%d %H:%M:%S.%f"
now_utc = datetime.now(timezone('UTC'))
now_asia = now_utc.astimezone(timezone('Asia/Taipei'))
filesavename=str(now_asia.strftime(format))+' single_channel_'+str(channel_to_read)+'.txt'
filetosave=rootpath+filesavename
f = open(filetosave, 'w')
for led in base.leds:
        led.on()
print("start")
while (base.buttons[3].read()==0):
    now_utc = datetime.now(timezone('UTC'))
    now_asia = now_utc.astimezone(timezone('Asia/Taipei'))
    data_array[0]=str(now_asia.strftime(formatdata))+'\t'
    data_array[1]=str(ConvertIntan(channel_to_read))+'\n'
    f.writelines(data_array)
f.close()
print("finish")
for led in base.leds:
    led.off()

In [ ]:
#simple data viewer
y = []
#change '/home/xilinx/jupyter_notebooks/DATAtoRead.txt'
for line in open('2024-10-22 00:42:21 single_channel_8.txt','r'): 
    lines = [i for i in line.split('\t')] 
    #for single channel-->lines[1]
    #for multi channel-->lines[3]~line[18] corresponds to channel 8~23 can be used 
    y.append(float(lines[1]))
plt.plot(y) 
plt.show()